In [1]:
import pandas as pd
import numpy as np

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import statsmodels.api as sm

#from skbio.stats.composition import clr
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.decomposition import PCA
from factor_analyzer import FactorAnalyzer

# unumHelpers.py should be in the same folder as this ipynb file
#from unumHelpers import unumOLS


def color_big_red(val):
    color = 'red' if abs(val) >0.4 else 'black'
    return 'color: %s' % color

In [9]:
data = pd.read_excel('2020_CO_terms_cleaned.xlsx')

In [10]:
pca_cats = pd.read_excel('2020 Dems Category Mapping.xlsx')
dependents = pd.read_excel('2020 Dependent.xlsx')

In [11]:
dependents = dependents.rename(columns = {'Unnamed: 0':'Date'})

## Top_20 Campaign Finance 

In [12]:
top_20 = ['Date','tax','social security', 'funding', 'sam', 'taxes', 'cut', 
          'spending', 'funds', 'tax credit', 'cuts', 'spending',
          'funds','tax credit','cuts','income tax','shutdown','bureaucracy',
          'relief','cutting','government shutdown']

In [13]:
reg_df = pd.merge(data,pca_cats,on='Category',how='left').groupby('Term').sum().T
reg_df = reg_df.drop('Unnamed: 0', axis = 0)
reg_df = reg_df.reset_index().rename(columns={'index':'Date'})
reg_df['Date'] = pd.to_datetime(reg_df['Date'], infer_datetime_format=True)
reg_df = reg_df.filter(top_20)
reg_df = pd.merge(reg_df,dependents,on='Date',how='inner')
reg_df = reg_df.fillna(0)

In [14]:
reg_df.head()

,Date,tax,social security,funding,sam,taxes,cut,spending,funds,tax credit,...,income tax,shutdown,bureaucracy,relief,cutting,government shutdown,Hickenlooper_raw,Gardner_raw,Hickenlooper_adj,Gardner_adj
0,2019-02-15,0.007252,0.001951,0.001456,0.000408,0.001033,0.000233,0.000874,0.000231,0.000444,...,0.000336,0.004369,0.000175,0.000231,0.000175,0.003495,0.431907,0.568093,0.474177,0.525823
1,2019-02-16,0.006785,0.000835,0.001066,0.000522,0.001392,0.000261,0.000381,0.000227,0.000533,...,0.000477,0.001566,0.000065,0.000159,0.000087,0.002088,0.431490,0.568510,0.474032,0.525968
2,2019-02-17,0.006269,0.001112,0.000590,0.000708,0.001491,0.000303,0.000433,0.000142,0.000351,...,0.000325,0.000910,0.000137,0.000179,0.000202,0.000961,0.430931,0.569069,0.473836,0.526164
3,2019-02-18,0.006408,0.002341,0.000674,0.000469,0.001052,0.000329,0.000359,0.000259,0.000447,...,0.000334,0.000698,0.000102,0.000197,0.000164,0.000698,0.430737,0.569263,0.473768,0.526232
4,2019-02-19,0.005578,0.002305,0.000651,0.000458,0.000996,0.000332,0.000251,0.000240,0.000443,...,0.000398,0.000598,0.000132,0.000225,0.000199,0.000465,0.432308,0.567692,0.474318,0.525682


In [15]:
y = reg_df[['Gardner_adj','Hickenlooper_adj']]
reg_df.corr()[['Gardner_adj','Hickenlooper_adj']]

,Gardner_adj,Hickenlooper_adj
tax,0.475647,-0.475647
social security,-0.236621,0.236621
funding,0.134460,-0.134460
sam,-0.308519,0.308519
taxes,0.590531,-0.590531
cut,-0.201925,0.201925
spending,0.343867,-0.343867
funds,0.032407,-0.032407
tax credit,0.508917,-0.508917
cuts,-0.132408,0.132408
